In [1]:
#imports
import numpy as np
import pandas as pd
import time
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from sklearn.metrics import mean_squared_error
from math import sqrt
import math

In [2]:
# initialize variable
K = 2

In [3]:

rate= pd.read_csv('rating.txt', encoding="utf-8", sep="\t",
                         names=["item", "user", "rate", "status", "time", "edit", "type" , "vertical"])
print('rate data loaded')
rate = rate.drop(columns=['status', 'type' , 'vertical'])

trust= pd.read_csv('trust.txt', encoding="utf-8", sep="\t",
                         names=["user1", "user2", "trust", "time"])
print('trust data loaded')
trust['time'] =  pd.to_datetime(trust['time'], format='%Y-%m-%d')
rate['time'] =  pd.to_datetime(rate['time'], format='%Y-%m-%d')
rate['edit'] =  pd.to_datetime(rate['edit'], format='%Y-%m-%d')
print('time type set')
rate.loc[((pd.isnull(rate['edit']) ==False)), 'time'] = rate['edit']
print('editted fixed')
rate = rate.sort_values(by=['time'])
trust = trust.sort_values(by=['time'])
rate_time_gp = rate.groupby('time')
trust_time_gp = trust.groupby('time')
trust = trust.loc[trust['trust']==1]

rate data loaded
trust data loaded
time type set
editted fixed


In [10]:
def simpleMF_final(user_arr, item_arr,rate_dic,
                                P_dic, Q_dic, steps=70, K = 2,
                                   landa=0.0002, gama=0.002):
    print ('users :',len(user_arr),'items :', len(item_arr),'P :', len(P_dic),'Q :',len(Q_dic))

    Rui_dic={}
    start = 0 

    for step in range(steps):
        if step%10==0 or step ==49:
            print ('step = ',step, 'time = ' ,time.time()-start )

        start = time.time()

        for u in range(len(user_arr)):
            Rui_dic[user_arr[u]] = {}
            sigma = 0 

            for i in Q_dic.keys():
                if i in rate_dic[user_arr[u]]:
                    Pu = P_dic[user_arr[u]]
                    Qi = Q_dic[i]
                    sigma+= ( np.dot(Pu,Qi) - rate_dic[user_arr[u]][i]) * Qi
            P_dic[user_arr[u]] = P_dic[user_arr[u]] - landa * (sigma + gama *  P_dic[user_arr[u]] )

        # for item

        for i in range(len(item_arr)):
            sigma = 0 
            for u in P_dic.keys():
                if item_arr[i] in rate_dic[u]:
                    Pu = P_dic[u]
                    Qi = Q_dic[item_arr[i]]
                    sigma += ( np.dot(Pu,Qi) - rate_dic[u][item_arr[i]] ) * Pu
            Q_dic[item_arr[i]] = Q_dic[item_arr[i]] - landa * (sigma + gama * Q_dic[item_arr[i]])
            
        e = 0
        if step%10==0 or step ==49:
            e = 0
            ss=0.0
            n=0.0
            for u in range(len(user_arr)):
                for i in range(len(item_arr)):
                    if item_arr[i] in rate_dic[user_arr[u]]:
                        Pu = P_dic[user_arr[u]]
                        Qi = Q_dic[item_arr[i]]
                        e = e + pow(rate_dic[user_arr[u]][item_arr[i]] - np.dot(Pu,Qi), 2)
                        n+=1

            try:
                print('error = ',sqrt(e/n))
            except:
                print('ERROR NOT VALID!! ')

    return P_dic,Q_dic

In [ ]:
times = rate.time.unique()
P_dic={}
Q_dic={}
min_rate_dic={}
rate_dic={}
min_rate_item_dic={}
trust_user_dictionary = {}
    
        
for i in range(len(times)):
    print ('------------------------------- '+ str(times[i]) +', '+ str(i) +', '+ str(len(times)) +' ---------------------------------')
    
    
          
    if i==0:
        first_rate = rate.loc[(rate['time']<=times[i])]
        i+=1
    else :
        first_rate = rate.loc[((rate['time']<=times[i]) & (rate['time']>times[i-1]))]
        print ('calculate RMSE')    
        arr_Arr = []
        ss=0.0
        n=0.0
        for index, row in first_rate.iterrows():
            if row['user'] in P_dic and row['item'] in Q_dic:
                ss+=pow((row['rate'] - (np.dot(P_dic[row['user']],Q_dic[row['item']]))), 2)
                n+=1
        try :
            rmse = sqrt(ss/n)
        except :
            rmse = '!! NOT VALID, ' + str(n) + '!!'
        print ('total rmse = ' , rmse ,', numebrs = ' ,n)
    
    user_list = list(first_rate.user.unique())
    for u in user_list:
        if u not in P_dic:
            rate_dic[u]={}
    user_list.sort()
    
    item_list = list(first_rate.item.unique())
    
    item_list.sort()
    
    N = len(user_list)
    M = len(item_list)
    P = np.random.rand(N, K) + 0.5
    Q = np.random.rand(M, K) + 0.5
    
    mean_p = np.array(list(P_dic.values())).mean(axis=0)
    mean_q = np.array(list(Q_dic.values())).mean(axis=0)

    for index, row in first_rate.iterrows():
        if row['user'] in user_list and row['item'] in item_list:
            rate_dic[row['user']][row['item']] = row['rate']
    for user in range(len(P)):
        if i>1:
            P_dic[user_list[user]]= mean_p
        else :
            P_dic[user_list[user]]= P[user]
    for item in range(len(Q)):
        if i>1:
            Q_dic[item_list[item]]= mean_q
        else:       
            Q_dic[item_list[item]]= Q[item]
    
    P_dic,Q_dic = simpleMF_final(user_list,item_list,rate_dic,
                                       P_dic, Q_dic)
    np.save('PandQ_with_out_trust_MF/P_'+str(i)+'_'+ str(times[i]).partition('T')[0] +'.npy', P_dic)  
    np.save('PandQ_with_out_trust_MF/Q_'+str(i)+'_'+ str(times[i]).partition('T')[0] +'.npy', Q_dic)


------------------------------- 2001-01-10T00:00:00.000000000, 0, 906 ---------------------------------


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: Mean of empty slice.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: Mean of empty slice.


users : 1515 items : 219 P : 1515 Q : 219
step =  0 time =  1567091343.6643455
error =  2.6466210599077433
step =  10 time =  0.15664434432983398
error =  2.14474804096668
step =  20 time =  0.16059350967407227
error =  1.9006755753780011
step =  30 time =  0.15752792358398438
error =  1.7374735271570916
step =  40 time =  0.15809416770935059
error =  1.6104638426922455
step =  49 time =  0.15796732902526855
error =  1.5150515919923297
step =  50 time =  0.2938868999481201
error =  1.5053050972238873
step =  60 time =  0.19576382637023926
error =  1.4155255246734666
------------------------------- 2001-01-11T00:00:00.000000000, 1, 906 ---------------------------------
calculate RMSE
total rmse =  !! NOT VALID, 0.0!! , numebrs =  0.0
users : 1 items : 1 P : 1515 Q : 220
step =  0 time =  1567091358.2076313
error =  1.7048077718801564
step =  10 time =  0.0013632774353027344
error =  1.68495197407612
step =  20 time =  0.0013439655303955078
error =  1.665206226529682
step =  30 time =  0

error =  1.1391267238534504
step =  10 time =  35.91528391838074
error =  0.7436293339749919
step =  20 time =  36.73382377624512
error =  1.3269480921932175
step =  30 time =  36.748717069625854
error =  1.8544157712873521
step =  40 time =  35.60431408882141
error =  1.45724645570894
step =  49 time =  36.490278482437134
error =  1.484423787178169
step =  50 time =  38.84542632102966
error =  1.3898823383997525
step =  60 time =  35.78950500488281
error =  1.3578140936962766
------------------------------- 2001-01-26T00:00:00.000000000, 12, 906 ---------------------------------
calculate RMSE
total rmse =  2.4883541229423876 , numebrs =  7231.0
users : 6729 items : 6254 P : 24137 Q : 42254
step =  0 time =  1567117438.8469937
error =  1.1589850461216553
step =  10 time =  72.41955542564392
error =  3.732981891015119
step =  20 time =  73.294837474823
error =  1.8621608049345577
step =  30 time =  74.65067458152771
error =  1.79544028491319
step =  40 time =  73.5125572681427
error = 

error =  1.8063697348763592
step =  30 time =  98.15247797966003
error =  1.7163577594553112
step =  40 time =  101.10948157310486
error =  1.6957223462722422
step =  49 time =  91.61236667633057
error =  1.5690067148456888
step =  50 time =  95.36493062973022
error =  1.509031732502657
step =  60 time =  92.53377199172974
error =  1.3153201716588796
------------------------------- 2001-02-06T00:00:00.000000000, 23, 906 ---------------------------------
calculate RMSE
total rmse =  2.7133185499828394 , numebrs =  11612.0
users : 6490 items : 5572 P : 33270 Q : 77783
step =  0 time =  1567190991.478289
error =  1.1962586416140648
step =  10 time =  131.30647492408752
error =  2.4408622370470274
step =  20 time =  123.00098180770874
error =  1.7817970111220232
step =  30 time =  135.42324876785278
error =  1.7638466844975667
step =  40 time =  123.30977725982666
error =  1.723236779170464
step =  49 time =  128.84886860847473
error =  1.573231121810083
step =  50 time =  127.402331590652

...

error =  3.0745944141766643
step =  60 time =  299.1195294857025
error =  2.516219184427079
------------------------------- 2001-03-22T00:00:00.000000000, 67, 906 ---------------------------------
calculate RMSE
total rmse =  1.98033055835641 , numebrs =  10263.0
users : 3222 items : 6196 P : 51925 Q : 186837
step =  0 time =  1567822051.2501972
error =  1.2504074956251776
step =  10 time =  172.47731065750122
error =  2.944486707717366
step =  20 time =  171.45312690734863
error =  2.292196114242621
step =  30 time =  264.43837571144104
error =  1.7470126411914533
step =  40 time =  255.62857866287231
error =  1.511603533205405
step =  49 time =  232.20371460914612
error =  1.5419732815803298
step =  50 time =  231.68193984031677
error =  1.4604719998284177
step =  60 time =  291.9832580089569
error =  1.577333304574676
------------------------------- 2001-03-23T00:00:00.000000000, 68, 906 ---------------------------------
calculate RMSE
total rmse =  2.374539934986258 , numebrs =  12

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in multiply
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in multiply
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in subtract


step =  40 time =  299.82499957084656
error =  nan
step =  49 time =  301.8790657520294
error =  nan
step =  50 time =  311.38331604003906
error =  nan
step =  60 time =  403.99667406082153
error =  nan
------------------------------- 2001-03-25T00:00:00.000000000, 70, 906 ---------------------------------
calculate RMSE
total rmse =  nan , numebrs =  22317.0
users : 6066 items : 7236 P : 53037 Q : 194746
step =  0 time =  1567886507.6368697
error =  nan
step =  10 time =  311.7414815425873
error =  nan
step =  20 time =  286.14709424972534
error =  nan
step =  30 time =  401.6106128692627
error =  nan
step =  40 time =  302.87735295295715
error =  nan
step =  49 time =  393.9099316596985
error =  nan
step =  50 time =  412.8900725841522
error =  nan
step =  60 time =  433.64371943473816
error =  nan
------------------------------- 2001-03-26T00:00:00.000000000, 71, 906 ---------------------------------
calculate RMSE
total rmse =  nan , numebrs =  23678.0
users : 10900 items : 9404 P 

total rmse =  nan , numebrs =  8613.0
users : 6339 items : 2256 P : 57208 Q : 223660
step =  0 time =  1568246298.522044
error =  nan
step =  10 time =  276.52951431274414
error =  nan
step =  20 time =  349.74595522880554
error =  nan
step =  30 time =  347.5432593822479
error =  nan
step =  40 time =  346.8844678401947
error =  nan
step =  49 time =  349.4150640964508
error =  nan
step =  50 time =  350.20355319976807
error =  nan
step =  60 time =  319.2980740070343
error =  nan
------------------------------- 2001-04-08T00:00:00.000000000, 84, 906 ---------------------------------
calculate RMSE
total rmse =  nan , numebrs =  6345.0
users : 2451 items : 3464 P : 57288 Q : 224605
step =  0 time =  1568269681.9392865
error =  nan
step =  10 time =  205.98619651794434
error =  nan
step =  20 time =  213.53618478775024
error =  nan
step =  30 time =  212.36583185195923
error =  nan
step =  40 time =  205.52649450302124
error =  nan
step =  49 time =  204.89603185653687
error =  nan
ste

step =  40 time =  393.2095522880554
error =  nan
step =  49 time =  348.6753695011139
error =  nan
step =  50 time =  379.8239426612854
error =  nan
step =  60 time =  261.7625389099121
error =  nan
------------------------------- 2001-04-21T00:00:00.000000000, 97, 906 ---------------------------------
calculate RMSE
total rmse =  nan , numebrs =  11629.0
users : 4465 items : 5301 P : 60375 Q : 248750
step =  0 time =  1568597110.1226234
error =  nan
step =  10 time =  293.1174077987671
error =  nan
step =  20 time =  324.92778730392456
error =  nan
step =  30 time =  300.8311777114868
error =  nan
step =  40 time =  324.25362610816956
error =  nan
step =  49 time =  410.465078830719
error =  nan
step =  50 time =  413.12005138397217
error =  nan
step =  60 time =  260.7910952568054
error =  nan
------------------------------- 2001-04-22T00:00:00.000000000, 98, 906 ---------------------------------
calculate RMSE
total rmse =  nan , numebrs =  9681.0
users : 2620 items : 4766 P : 6049